# Principal Component Analysis 

PCA is a dimensionality reduction technique; it lets you distill multi-dimensional data down to fewer dimensions, selecting new dimensions that preserve variance in the data as best it can.

We Will See How To Implement it in this python notebook with the diffrent way: Using Our Pure Code , With Numpy and With Sklearn library But First Let's Take a Look How PCA Work Under the hood.

## PCA Approach 

1. Standardize the data.
- Obtain the Eigenvectors and Eigenvalues from the covariance matrix or correlation matrix, or perform Singular Vector   Decomposition.
- Sort eigenvalues in descending order and choose the $k$ eigenvectors that correspond to the $k$ largest eigenvalues where $k$ is the number of dimensions of the new feature subspace $(k≤d)$.
- Construct the projection matrix $W$ from the selected $k$ eigenvectors.
- Transform the original dataset $X$ via $W$ to obtain a $k$-dimensional feature subspace $Y$.


### 1. Standardizing The Data

Most of the times, our dataset will contain features highly varying in magnitudes, units and range. But since, most of the machine learning algorithms use Eucledian distance between two data points in their computations, this is a problem.

If left alone, these algorithms only take in the magnitude of features neglecting the units. The results would vary greatly between different units, 5kg and 5000gms. The features with high magnitudes will weigh in a lot more in the distance calculations than features with low magnitudes.

Whether to standardize the data prior to a PCA on the covariance matrix depends on the measurement scales of the original features. Since PCA yields a feature subspace that maximizes the variance along the axes, it makes sense to standardize the data, especially, if it was measured on different scales.

<img src = "https://cdn-images-1.medium.com/max/1600/1*EyPd0sQxEXtTDSJgu72JNQ.jpeg" title ="Tiny Features vs Mega Features"/>

### 2. Computing Eigenvectors and Eigenvalues

The eigenvectors and eigenvalues of a covariance (or correlation) matrix represent the "core" of a PCA: The eigenvectors (principal components) determine the directions of the new feature space, and the eigenvalues determine their magnitude. In other words, the eigenvalues explain the variance of the data along the new feature axes.

#### Covariance Matrix

The classic approach to PCA is to perform the eigendecomposition on the covariance matrix $Σ$, which is a $n×n$ matrix where each element represents the covariance between two features. The covariance between two features can be calculated like the following:

$\sigma_{jk}=\frac{1}{n−1}\sum_{i = 1}^N = (x_{ij}−\bar{x_{j}})(x_{ik}−\bar{x_{k}})$.

We can summarize the calculation of the covariance matrix via the following matrix equation:
$\sum_=\frac{1}{n−1}((X−\bar{x})^T(X−\bar{x}))$

where $\bar{x}$ is the mean vector $\bar{x}=\sum_{i = 1}^n x_{i}.$

The mean vector is a d-dimensional vector where each value in this vector represents the sample mean of a feature column in the dataset.


#### EigenValues and EigenVectors

In essence, an eigenvector v of a linear transformation T is a non-zero vector that, when T is applied to it, does not change direction. Applying T to the eigenvector only scales the eigenvector by the scalar value λ, called an eigenvalue. This condition can be written as the equation ${\displaystyle T(\mathbf {v} )=\lambda \mathbf {v} ,} $ then the eigenvalue equation above for a linear transformation can be rewritten as the matrix multiplication ${\displaystyle Av=\lambda v} $

then v is an eigenvector of the linear transformation A and the scale factor λ is the eigenvalue corresponding to that eigenvector. Equation is the eigenvalue equation for the matrix A. Equation  can be stated equivalently as ${\displaystyle (A-\lambda I)v=0}$ where I is the n by n identity matrix.

Equation has a non-zero solution $v$ if and only if the determinant of the matrix $(A − λI)$ is zero. Therefore, the eigenvalues of $A$ are values of $λ$ that satisfy the equation ${\displaystyle |A-\lambda I|=0}$


### 3. Sort eigenvalues
The typical goal of a PCA is to reduce the dimensionality of the original feature space by projecting it onto a smaller subspace, where the eigenvectors will form the axes.

In order to decide which eigenvector(s) can dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues: The eigenvectors with the lowest eigenvalues bear the least information about the distribution of the data; those are the ones can be dropped.
In order to do so, the common approach is to rank the eigenvalues from highest to lowest in order choose the top $k$ eigenvectors.

### 4. Construct the projection matrix

It's about time to get to the really interesting part: The construction of the projection matrix that will be used to transform the dataset onto the new feature subspace. Although, the name "projection matrix" has a nice ring to it, it is basically just a matrix of our concatenated top $k$ eigenvectors.

### 5.  Transform the original dataset

In this last step we will use the $d*k$-dimensional projection matrix $W$ to transform our samples onto the new subspace via the equation
$Y=X×W$, where $Y$ is matrix of our transformed samples.

# PCA Using Numpy

In [ ]:
# Importing The Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# getting the data from csv file
data = pd.read_csv('Wine.csv')
data.head()

In [ ]:
# Spliting Them to Features and targets
X = data.iloc[:, 0:13].values
y = data.iloc[:, 13].values


In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
# Calculating Covariance Matrix
cov_matrix = np.cov(X.T)

In [ ]:
# Getting the Eign Values And EigenVecorts
eig_vals, eig_vecs = np.linalg.eig(cov_matrix)

In [ ]:
# Forming a pair of Eign Values / Pairs
eig_pairs = [(np.abs(eig_vals[i]) , eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sorting All of Them
eig_pairs.sort(key = lambda x : x[0] , reverse= True)

In [ ]:
final = []
for i in range(2):
    final.append(eig_pairs[i][1].reshape(13,1))

# Creating the Projection Matrix
matrix_w = np.hstack((final))

In [ ]:
# Transforming The Data
Y = X.dot(matrix_w)

In [ ]:
# Ploting The Data
from matplotlib.colors import ListedColormap

for i, j in enumerate(np.unique(y)):
    plt.scatter(Y[y == j, 0], Y[y == j, 1],
                c = ListedColormap(('red', 'green', 'blue'))(i), label = j)
plt.title('Customer_Segment Wine')
plt.legend()
plt.show()

# PCA Using Sklearn

In [ ]:
# Getting The PCA From sklearn library
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
New = pca.fit_transform(X)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

In [ ]:
# Ploting The Data For Sklearn Result
from matplotlib.colors import ListedColormap

for i, j in enumerate(np.unique(y)):
    plt.scatter(New[y == j, 0], New[y == j, 1],
                c = ListedColormap(('red', 'green', 'blue'))(i), label = j)
plt.title('Customer_Segment Wine')
plt.legend()
plt.show()

# PCA Purly
In This Section We Gonna Implement PCA Using Our Own Algorithm.

In [ ]:
#Matrix multiplication
def matmul (A, B):
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    if cols_A != rows_B:
        print('Cannot multiply the two matrices. Incorrect dimensions.')
        return

    C = [[0 for row in range(cols_B)] for col in range(rows_A)]
    
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                C[i][j] += A[i][k] * B[k][j]
    return C

# simple mean function
def mean(a_list):
    return sum(a_list)/len(a_list)

# calculate the variance of a list
def variance(a_list):
    mean_value = mean(a_list)
    squared = [(x-mean_value)**2 for x in a_list]
    return sum(squared) / len(a_list)

# calculate standard deviation of a vetor
def std(a_list):
    return (variance(a_list)**(1/2))

# Standardization of a vector x = x - mean(x) / std(x)
def stand_vactor(a):
    result = []
    meanA = mean(a)
    stdA = std(a)
    for i in a:
        result.append((i - meanA)/ stdA)
    return result 

# Standardization of a matrix x = x - mean(x) / std(x)
def stand_matrix(a):
    zipa= list(zip(*a))
    result = []
    for i in zipa:
        result.append(stand_vactor(i))
    return list(map(list,zip(*result)))

# Calculate the covariance matrix for a dataset
def covariance_matrix(matrix):
    pass

# calculate the eigen value and eigen vetcors of a matrix
def eigns_things(matrix):
    pass